# 1.3

### 練習問題1.29
---
**シンプソンの公式**

被積分関数$f(x)$を二次関数$P(x)$で近似することで成立する。

積分区間を$a$から$b$、$m = (a+b)/2$とする。

$$
P(x) = f(a) \frac{(x-m)(x-b)}{(a-m)(a-b)} + f(m) \frac{(x-a)(x-b)}{(m-a)(m-b)} + f(b) \frac{(x-a)(x-m)}{(b-a)(b-m)}
$$

このとき、

$$
\int_a^b f(x) dx \approx \int_a^b P(x)
$$

この式は積分区間が広いと誤差が大きくなる。

---
ここで使用してるのは***合成シンプソン公式***で、積分区間を小さく分割し、それぞれの区間でシンプソンの公式を用いたもの


$$
\int_a^b f(x) \approx \frac{h}{3} (y_0 + 4y_1 + 2y_2 \cdots 2y_{n-1} + 4y_{n-1} + y_n) 
$$


---



In [1]:
(define (cube x) (* x x x))

In [2]:
(define (sum term a next b)
  (if (> a b) 
      0
    (+ (term a)
        (sum term (next a) next b))))

In [3]:
(define (integral f a b dx)
  (define (add-dx x) (+ x dx))
  (* (sum f (+ a (/ dx 2.0)) add-dx b)
     dx))

In [4]:
(integral cube 0 1 0.01)

0.24998750000000042

In [5]:
(integral cube 0 1 0.001)

0.249999875000001

---

#### 解答 1.29

In [6]:
(define (simpson f a b n) ; f : function(term) / a,b : [a,b] / n : dx
  (define (sum term a next b) 
      (if (> a b) 
          0
        (+ (term a)
            (sum term (next a) next b))))
  (define h (/ (- b a) n))
  (define (add-h2 x) (+ x h h))
  (* (/ h 3.0)
     (- (+ (* 4.0 (sum f (+ a h) add-h2 b)) ; sum y_k (k : odd)
           (* 2.0 (sum f a add-h2 b))) ; sum y_k (k : even)
        (+ (f a) (f b)) ; 余分なところを引く
     )
  )
)

In [7]:
(simpson cube 0 1 10)

0.25

In [8]:
(simpson cube 0 1 100)

0.25

In [9]:
(simpson cube 1 2 100)

3.7500000000000004

In [10]:
(simpson cube (+ 1 (/ 1 2)) 2 100)

2.734375

In [11]:
(simpson cube 1 (+ 1 (/ 1 2)) 100)

1.015625

In [12]:
(+ 2.734375 1.015625)

3.75

In [13]:
(- 10 2 3)

5

### 練習問題 1.30
---

#### 解答1.30

In [14]:
(define (sum2 term a next b)
  (define (iter a result)
    (if (> a b)
        result
        (iter (next a) (+ (term a) result))))
    (iter a 0))

In [15]:
(define (simpson f a b n)
  (define h (/ (- b a) n))
  (define (add-h2 x) (+ x h h))
  (* (/ h 3.0)
     (- (+ (* 4.0 (sum2 f (+ a h) add-h2 b))
           (* 2.0 (sum2 f a add-h2 b)))
        (+ (f a) (f b))
     )))

In [16]:
(simpson cube 1 2 100)

3.7500000000000004

### 練習問題1.31
---

#### 解答1.31

In [17]:
(define (product f a b suc)
  (if (> a b)
      1
  (* (f a) 
     (product f (suc a) b suc))))

In [18]:
(define (suc n) (+ n 1))
(product cube 1 3 suc)

216

In [19]:
(define (factorial n)
  (define (suc m) (+ m 1))
  (define (f x) x)  
  (* 1 
     (product f 1 n suc)))

In [20]:
(factorial 6)

720

#### ウォリスの公式
$$
\frac{\pi}{2} = \prod_{i=1}^{\infty} (\frac{2n}{2n-1} \times \frac{2n}{2n+1}) = \frac{2\times2\times4\times4\times\cdots}{1\times3\times3\times5\times\cdots}
$$

In [21]:
(define (getPi iter)
  (define (suc2 n) (+ n 2.0))
  (define (f m) (* m (+ m 2.0)))
  (define (g m) (* m m))
  (* 2.0
     (/ (product g 2.0 iter suc2)
        (product f 1.0 iter suc2))
  ))

In [22]:
(getPi 170) ; 170 is limit

3.1324201790229065

In [23]:
(define (product2 f a b suc)
  (if (> a b)
      1
      (* (f a)
         (product2 f (suc a) b suc))))

In [24]:
(define (suc n) (+ n 1))
(product2 cube 1 3 suc)

216

### 練習問題 1.32
---

In [25]:
(define (accumulate combiner null-value term a next b)
  (if (> a b)
      null-value
      (combiner (term a)
                (accumulate combiner null-value term (next a) next b))))

In [26]:
(define (a_sum term a next b)
  (accumulate + 0 term a next b))

In [27]:
(define (p1 n) (+ n 1))
(a_sum cube 1 p1 2)

9

In [28]:
(define (a_product term a next b)
  (accumulate * 1 term a next b))

In [29]:
(a_product cube 1 p1 3)

216

---

In [30]:
(define (accumulate2 combiner null-value term a next b)
  (define (iter a result)
    (if (> a b)
        null-value
        (combiner (term a)
                  (iter (next a) result))))
  (iter a null-value))

In [31]:
(define (a_product2 term a next b)
  (accumulate2 * 1 term a next b))

In [32]:
(a_product2 cube 1 p1 3)

216

### 練習問題 1.33
---

In [33]:
(define (filtered-accumulate combiner null-value term a next b filter)
  (if (> a b)
   null-value
  (combiner (if (filter a)
            (term a)
            null-value)
            (filtered-accumulate combiner null-value term (next a) next b filter))))

In [34]:
(define (accumulate combiner null-value term a next b)
  (if (> a b)
      null-value
      (combiner (term a)
                (accumulate combiner null-value term (next a) next b))))

In [35]:
(define (square x) (* x x))
(define (smallest-divisor n) (find-divisor n 2)) (define (find-divisor n test-divisor)
(cond ((> (square test-divisor) n) n)
((divides? test-divisor n) test-divisor) (else (find-divisor n (+ test-divisor 1)))))
(define (divides? a b) (= (remainder b a) 0))
(define (prime? n)
(= n (smallest-divisor n)))

(define (next n) (+ n 1))
(define (add a b)  (+ a b))

In [36]:
(define (p-square a b)
  (filtered-accumulate + 0 square a next b prime?))

In [37]:
(p-square 2 5)

38